In [1]:
# https://towardsdatascience.com/how-to-use-the-staple-algorithm-to-combine-multiple-image-segmentations-ce91ebeb451e
# packages
import nibabel as nib # https://nipy.org/nibabel/
import SimpleITK as sitk # https://simpleitk.org/
from matplotlib import pyplot as plt
import os
from meidic_vtach_utils.run_on_recommended_cuda import get_cuda_environ_vars as get_vars
os.environ.update(get_vars(select="*"))
from collections import OrderedDict
import torch
import numpy as np


### Recommended gpus on this machine (descending order) ###
  ID  Card name                     Util    Mem free  Cuda             User(s)
----  --------------------------  ------  ----------  ---------------  ---------
   0  Tesla T4                       0 %   14907 MiB  11.6(510.39.01)
   1  NVIDIA GeForce RTX 2080 Ti     0 %   11015 MiB  11.6(510.39.01)
   3  NVIDIA GeForce RTX 2080 Ti     0 %   11015 MiB  11.6(510.39.01)
   4  NVIDIA GeForce RTX 2080 Ti     0 %   11015 MiB  11.6(510.39.01)
   5  NVIDIA GeForce RTX 2080 Ti     0 %   11015 MiB  11.6(510.39.01)
   2  Quadro RTX 8000               35 %   29924 MiB  11.6(510.39.01)  weihsbach

Will apply following mapping

  ID  Card name        torch
----  -----------  --  -------
   0  Tesla T4     ->  cuda:0


In [2]:
DATA_PATH = "/share/data_supergrover1/weihsbach/shared_data/important_data_artifacts/curriculum_deeplab/20220114_crossmoda_multiple_registrations/crossmoda_deeds_registered.pth"
bare_data = torch.load(DATA_PATH)

In [8]:
reg_state = "acummulate_deeds_FT2_MT1"

staple_filter = sitk.STAPLEImageFilter()
# sitk.ProcessObject.SetGlobalDefaultDebugOff()
FOREGROUND = 1.0
weight_data = {}
EVERY = 1
staple_filter.SetForegroundValue(FOREGROUND)
DEBUG = False
if reg_state == "acummulate_deeds_FT2_MT1":
    for fixed_id, moving_dict in bare_data.items():
        # if fixed_id != '111l': continue
        print(fixed_id)
        # print(moving_dict)
        sorted_moving_dict = OrderedDict(moving_dict)
        for slice_idx in range(45,95):
            # slice_idx = 64
            print(slice_idx)
            moving_data = []
            selected_moving_ids = []
            # staple_filter = sitk.STAPLEImageFilter()
            # staple_filter.SetForegroundValue(FOREGROUND)

            for idx_mov, (moving_id, moving_sample) in enumerate(sorted_moving_dict.items()):
                # Only use every third warped sample
                print(idx_mov)
                # idx_mov = 29
                if idx_mov % EVERY == 0:
                    moving_data.append(moving_sample['warped_label'][slice_idx].cpu())
                    moving_slice_id = f"{fixed_id}:m{moving_id}W{slice_idx-45:03d}"
                    selected_moving_ids.append(moving_slice_id)

            sitk_moving_data = [sitk.GetImageFromArray(reg_seg.to_dense().numpy().astype(np.int16)) for reg_seg in moving_data]
            _ = staple_filter.Execute(sitk_moving_data)
            # del staple_filter
            # del sitk_moving_data
            # staple_out = sitk.STAPLE(sitk_moving_data, FOREGROUND)
            # consensus = sitk.GetArrayFromImage(staple_out)
            
            # specitivity = staple_filter.GetSpecificity()
            # sensitivity = staple_filter.GetSensitivity()
            # specitivity = sensitivity = [0]*60
            # f_weight_dict = weight_data.get(fixed_id, {})
            # staple_consensus = sitk.GetArrayFromImage(staple_out)
            # for moving_id, sens, spec in zip(selected_moving_ids, sensitivity, specitivity):
            #     weight_data[moving_id] = dict(sensitivity=sens, specitivity=spec)
            # weight_data[fixed_id] = f_weight_dict
        
        if DEBUG: break
else:
    raise ValueError()

# weight_data['data_path'] = DATA_PATH
# torch.save(weight_data, f"./data/staple_calc/{reg_state}_every_{EVERY}_2dslices.pth")


108r
45
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
46
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
47
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
48
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
49
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
50
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
51
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
52
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
53
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
54
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
55
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
56
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29

In [ ]:
dps = []
for key in weight_data.keys():
    if key == 'data_path': continue
    if np.isnan(weight_data[key]['sensitivity']):
        pass
    else:
        dps.append(weight_data[key]['sensitivity'])
plt.hist(dps)

In [ ]:
network_scores = torch.load("/share/data_supergrover1/weihsbach/shared_data/tmp/curriculum_deeplab/data/output/noble-forest-1144_fold0_epx39/train_label_snapshot.pth")

In [ ]:
d_ids = network_scores['d_ids']
network_parameter = network_scores['data_parameters']

In [ ]:
set(d_ids) == set(weight_data.keys())

In [ ]:
print(sorted(d_ids)[:10])
print(sorted(weight_data.keys())[:10])
bare_data.keys()



In [ ]:
print(list(weight_data.keys()))

# READ stapled samples

In [26]:
from glob import glob
import re
import nibabel as nib
staple_descri = glob("/share/data_supergrover1/heinrich/crossmoda_deeds/Fcrossmoda_*_L_staple*.nii.gz")
stapled_files = glob("/share/data_supergrover1/heinrich/crossmoda_deeds/Fcrossmoda_*_L_staple*.nii.gz")

staple_dict = {}
target_t2_keys_w_tumour = ['108r', '111l', '112r', '115l', '118r', '120r', '117l', '123r', '127r', '125l', '134r', '135r', '126l', '142r', '144r', '133l', '148r', '154r', '136l', '160r', '165r', '140l', '166r', '167r', '141l', '168r', '171r', '143l', '173r', '174r', '145l', '179r', '146l', '180r', '181r', '185r', '147l', '195r', '149l', '198r', '204r', '152l', '205r', '209r', '158l', '210r', '162l', '164l', '175l', '177l', '178l', '183l', '187l', '188l', '189l', '190l', '192l', '199l', '200l', '202l']

for _file in stapled_files:
    for staple_split in ['01','23','45','689']:
    # result = re.findall(r'/share/data_supergrover1/heinrich/crossmoda_deeds/Fcrossmoda_(\w+)_L_staple([0-9]+)\.nii\.gz', _file)
        result = re.findall(rf'/share/data_supergrover1/heinrich/crossmoda_deeds/Fcrossmoda_(\w+)_L_staple{staple_split}\.nii\.gz', _file)
        if not result: 
            continue
        else:
            fixed_num_str = result[0]
        
        "/share/data_supergrover1/heinrich/crossmoda_deeds/Fcrossmoda_*_L_staple{01,23,45,689}.nii.gz"
        if fixed_num_str+'r' in target_t2_keys_w_tumour:
            fixed_id = fixed_num_str+'r'
        elif fixed_num_str+'l' in target_t2_keys_w_tumour:
            fixed_id = fixed_num_str+'l'
        split_dict = staple_dict.get(fixed_id, {})
        split_dict[staple_split] = dict(
            warped_label=torch.tensor(nib.load(_file).get_fdata()).long().to_sparse(),
            file_path=_file
        )
        staple_dict[fixed_id] = split_dict
        print(staple_split, fixed_id)
    
torch.save(staple_dict, "./crossmoda_staple_consensus.pth")

45 177l
689 177l
01 111l
23 111l
45 111l
689 111l
01 115l
23 115l
45 115l
689 115l
01 117l
23 117l
45 117l
689 117l
01 125l
23 125l
45 125l
689 125l
01 126l
23 126l
45 126l
689 126l
01 133l
23 133l
45 133l
689 133l
01 146l
23 146l
45 146l
689 146l
01 140l
23 140l
45 140l
689 140l
01 141l
23 141l
45 141l
689 141l
01 143l
23 143l
45 143l
689 143l
01 145l
23 145l
45 145l
689 145l
01 147l
23 147l
45 147l
689 147l
01 149l
23 149l
45 149l
689 149l
01 152l
23 152l
45 152l
689 152l
01 158l
23 158l
45 158l
689 158l
01 162l
23 162l
45 162l
689 162l
01 164l
23 164l
45 164l
689 164l
01 175l
23 175l
45 175l
689 175l
01 177l
23 177l
01 178l
23 178l
45 178l
689 178l
01 183l
23 183l
45 183l
689 183l
01 187l
23 187l
45 187l
689 187l
01 188l
23 188l
45 188l
689 188l
01 189l
23 189l
45 189l
689 189l
01 190l
23 190l
45 190l
689 190l
01 192l
23 192l
45 192l
689 192l
01 199l
23 199l
45 199l
689 199l
01 200l
23 200l
45 200l
689 200l
01 202l
23 202l
45 202l
689 202l


In [25]:
print(staple_dict['177r'])

{'45': {'warped_label': tensor(indices=tensor([[26, 26, 26,  ..., 66, 66, 67],
                       [76, 77, 77,  ..., 72, 72, 70],
                       [64, 62, 63,  ..., 70, 71, 71]]),
       values=tensor([1, 1, 1,  ..., 1, 1, 1]),
       size=(128, 128, 128), nnz=14135, layout=torch.sparse_coo), 'file_path': '/share/data_supergrover1/heinrich/crossmoda_deeds/Fcrossmoda_177_L_staple45.nii.gz'}, '689': {'warped_label': tensor(indices=tensor([[21, 21, 21,  ..., 66, 66, 66],
                       [63, 63, 64,  ..., 71, 71, 71],
                       [70, 71, 70,  ..., 70, 71, 72]]),
       values=tensor([1, 1, 1,  ..., 1, 1, 1]),
       size=(128, 128, 128), nnz=17524, layout=torch.sparse_coo), 'file_path': '/share/data_supergrover1/heinrich/crossmoda_deeds/Fcrossmoda_177_L_staple689.nii.gz'}, '01': {'warped_label': tensor(indices=tensor([[26, 26, 26,  ..., 64, 65, 65],
                       [67, 68, 69,  ..., 73, 69, 69],
                       [76, 76, 75,  ..., 65, 69, 70]]),